# Combining verification results

* This notebook combines verification results on individual initializaitons for visualization.
* The notebook runs with `verif_config_6h.yml` in the same folder.

In [1]:
import os
import sys
import yaml
import argparse
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu
import score_utils as su

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

### Load config

In [3]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = conf = yaml.safe_load(stream)

## ERA5 conservation residuals

In [4]:
base_dir = conf['ERA5_weatherbench']['save_loc_clim']
files_025_mass = base_dir + 'ERA5_mass_residual_025_full_{:05d}_{:05d}.nc'
files_025_water = base_dir + 'ERA5_water_residual_025_full_{:05d}_{:05d}.nc'
files_025_energy = base_dir + 'ERA5_energy_residual_025_full_{:05d}_{:05d}.nc'

files_1deg_mass = base_dir + 'ERA5_mass_residual_subset_{:05d}_{:05d}.nc'
files_1deg_water = base_dir + 'ERA5_water_residual_subset_{:05d}_{:05d}.nc'
files_1deg_energy = base_dir + 'ERA5_energy_residual_subset_{:05d}_{:05d}.nc'

IND_max = 4*(365+366)
INDs = np.arange(0, IND_max+100, 100)
INDs[-1] = IND_max

### 1 deg subsetted version

In [5]:
collect_mass_residual = []
collect_water_residual = []
collect_energy_residual = []

for i, ind_start in enumerate(INDs[:-1]):
    ind_end = INDs[i+1]
    collect_mass_residual.append(xr.open_dataset(files_1deg_mass.format(ind_start, ind_end)))
    collect_water_residual.append(xr.open_dataset(files_1deg_water.format(ind_start, ind_end)))
    collect_energy_residual.append(xr.open_dataset(files_1deg_energy.format(ind_start, ind_end)))

ds_mass_1deg = xr.concat(collect_mass_residual, dim='time')
ds_water_1deg = xr.concat(collect_water_residual, dim='time')
ds_energy_1deg = xr.concat(collect_energy_residual, dim='time')

### 0.25 deg 37 level version

In [6]:
collect_mass_residual = []
collect_water_residual = []
collect_energy_residual = []

for i, ind_start in enumerate(INDs[:-1]):
    ind_end = INDs[i+1]
    collect_mass_residual.append(xr.open_dataset(files_025_mass.format(ind_start, ind_end)))
    collect_water_residual.append(xr.open_dataset(files_025_water.format(ind_start, ind_end)))
    collect_energy_residual.append(xr.open_dataset(files_025_energy.format(ind_start, ind_end)))

ds_mass_025 = xr.concat(collect_mass_residual, dim='time')
ds_water_025 = xr.concat(collect_water_residual, dim='time')
ds_energy_025 = xr.concat(collect_energy_residual, dim='time')

## Forecast consevation residuals

### FuXi-base

In [7]:
base_dir = conf['fuxi_dry']['save_loc_verif']

files_mass = base_dir + 'fuxi_dry_mass_residual_subset_{:05d}_{:05d}.nc'
files_water = base_dir + 'fuxi_dry_water_residual_subset_{:05d}_{:05d}.nc'
files_energy = base_dir + 'fuxi_dry_energy_residual_subset_{:05d}_{:05d}.nc'

IND_max = 365+366
INDs = np.arange(0, IND_max+50, 50)
INDs[-1] = IND_max

In [8]:
collect_mass_residual = []
collect_water_residual = []
collect_energy_residual = []

for i, ind_start in enumerate(INDs[:-1]):
    ind_end = INDs[i+1]
    collect_mass_residual.append(xr.open_dataset(files_mass.format(ind_start, ind_end)))
    collect_water_residual.append(xr.open_dataset(files_water.format(ind_start, ind_end)))
    collect_energy_residual.append(xr.open_dataset(files_energy.format(ind_start, ind_end)))

ds_mass_dry = xr.concat(collect_mass_residual, dim='days')
ds_water_dry = xr.concat(collect_water_residual, dim='days')
ds_energy_dry = xr.concat(collect_energy_residual, dim='days')

### FuXi-physics

In [9]:
base_dir = conf['fuxi_physics']['save_loc_verif']

files_mass = base_dir + 'fuxi_physics_mass_residual_subset_{:05d}_{:05d}.nc'
files_water = base_dir + 'fuxi_physics_water_residual_subset_{:05d}_{:05d}.nc'
files_energy = base_dir + 'fuxi_physics_energy_residual_subset_{:05d}_{:05d}.nc'

IND_max = 365+366
INDs = np.arange(0, IND_max+50, 50)
INDs[-1] = IND_max

In [10]:
collect_mass_residual = []
collect_water_residual = []
collect_energy_residual = []

for i, ind_start in enumerate(INDs[:-1]):
    ind_end = INDs[i+1]
    collect_mass_residual.append(xr.open_dataset(files_mass.format(ind_start, ind_end)))
    collect_water_residual.append(xr.open_dataset(files_water.format(ind_start, ind_end)))
    collect_energy_residual.append(xr.open_dataset(files_energy.format(ind_start, ind_end)))

ds_mass_physics = xr.concat(collect_mass_residual, dim='days')
ds_water_physics = xr.concat(collect_water_residual, dim='days')
ds_energy_physics = xr.concat(collect_energy_residual, dim='days')

## Preparing plot data

In [11]:
data_PLOT = {}

model_name = 'ERA5_025'

data_PLOT[f'{model_name}_mass_residual'] = ds_mass_025['mass_residual'].values
data_PLOT[f'{model_name}_mass_value'] = ds_mass_025['mass_value'].values

data_PLOT[f'{model_name}_water_residual'] = ds_water_025['water_residual'].values
data_PLOT[f'{model_name}_water_tendency'] = ds_water_025['water_tendency'].values
data_PLOT[f'{model_name}_evapor'] = ds_water_025['evapor'].values
data_PLOT[f'{model_name}_precip'] = ds_water_025['precip'].values

data_PLOT[f'{model_name}_energy_residual'] = ds_energy_025['energy_residual'].values
data_PLOT[f'{model_name}_energy_tendency'] = ds_energy_025['energy_tendency'].values
data_PLOT[f'{model_name}_atmos_top'] = ds_energy_025['atmos_top'].values
data_PLOT[f'{model_name}_surf'] = ds_energy_025['surf'].values

model_name = 'ERA5_1deg'

data_PLOT[f'{model_name}_mass_residual'] = ds_mass_1deg['mass_residual'].values
data_PLOT[f'{model_name}_mass_value'] = ds_mass_1deg['mass_value'].values

data_PLOT[f'{model_name}_water_residual'] = ds_water_1deg['water_residual'].values
data_PLOT[f'{model_name}_water_tendency'] = ds_water_1deg['water_tendency'].values
data_PLOT[f'{model_name}_evapor'] = ds_water_1deg['evapor'].values
data_PLOT[f'{model_name}_precip'] = ds_water_1deg['precip'].values

data_PLOT[f'{model_name}_energy_residual'] = ds_energy_1deg['energy_residual'].values
data_PLOT[f'{model_name}_energy_tendency'] = ds_energy_1deg['energy_tendency'].values
data_PLOT[f'{model_name}_atmos_top'] = ds_energy_1deg['atmos_top'].values
data_PLOT[f'{model_name}_surf'] = ds_energy_1deg['surf'].values

model_name = 'fuxi_dry'

data_PLOT[f'{model_name}_mass_residual'] = ds_mass_dry['mass_residual'].values
data_PLOT[f'{model_name}_mass_value'] = ds_mass_dry['mass_value'].values

data_PLOT[f'{model_name}_water_residual'] = ds_water_dry['water_residual'].values
data_PLOT[f'{model_name}_water_tendency'] = ds_water_dry['water_tendency'].values
data_PLOT[f'{model_name}_evapor'] = ds_water_dry['evapor'].values
data_PLOT[f'{model_name}_precip'] = ds_water_dry['precip'].values

data_PLOT[f'{model_name}_energy_residual'] = ds_energy_dry['energy_residual'].values
data_PLOT[f'{model_name}_energy_tendency'] = ds_energy_dry['energy_tendency'].values
data_PLOT[f'{model_name}_atmos_top'] = ds_energy_dry['atmos_top'].values
data_PLOT[f'{model_name}_surf'] = ds_energy_dry['surf'].values

model_name = 'fuxi_physics'

data_PLOT[f'{model_name}_mass_residual'] = ds_mass_physics['mass_residual'].values
data_PLOT[f'{model_name}_mass_value'] = ds_mass_physics['mass_value'].values

data_PLOT[f'{model_name}_water_residual'] = ds_water_physics['water_residual'].values
data_PLOT[f'{model_name}_water_tendency'] = ds_water_physics['water_tendency'].values
data_PLOT[f'{model_name}_evapor'] = ds_water_physics['evapor'].values
data_PLOT[f'{model_name}_precip'] = ds_water_physics['precip'].values

data_PLOT[f'{model_name}_energy_residual'] = ds_energy_physics['energy_residual'].values
data_PLOT[f'{model_name}_energy_tendency'] = ds_energy_physics['energy_tendency'].values
data_PLOT[f'{model_name}_atmos_top'] = ds_energy_physics['atmos_top'].values
data_PLOT[f'{model_name}_surf'] = ds_energy_physics['surf'].values

In [16]:
save_name = conf['qsub']['plot_data_loc'] + 'Conservation_results.npy'
# np.save(save_name, data_PLOT)